# Large Dataset Operations Example

This notebook demonstrates how to use the `jsonldf` package with large datasets.
We'll create a dataset with 100,000 records, perform various operations, and measure performance.

In [1]:
import pandas as pd
import sys
import os
import numpy as np
from datetime import datetime
import time

# Add parent directory to path to import jsonldb
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

from jsonldb.jsonldf import (
    save_jsonldf, load_jsonldf, update_jsonldf,
    select_jsonldf, delete_jsonldf, lint_jsonldf
)

## Generate Large Dataset

Let's create a function to generate a large dataset with multiple columns.

In [2]:
def generate_large_dataset(num_records=100000):
    """Generate a large dataset with multiple columns.
    
    Args:
        num_records: Number of records to generate
    """
    # Generate sample data
    data = {
        'value': np.random.normal(100, 15, num_records),
        'category': np.random.choice(['A', 'B', 'C', 'D'], num_records),
        'score': np.random.uniform(0, 1, num_records),
        'count': np.random.poisson(5, num_records),
        'timestamp': pd.date_range(
            start=datetime(2024, 1, 1),
            periods=num_records,
            freq='1min'
        )
    }
    
    # Create DataFrame with string index
    df = pd.DataFrame(data)
    df.index = [f'record_{i:06d}' for i in range(num_records)]
    
    return df

## Create Helper Function for Timing

Create a function to measure operation execution time.

In [3]:
def time_operation(operation, *args, **kwargs):
    """Time an operation and return its result.
    
    Args:
        operation: Function to time
        *args: Positional arguments for the operation
        **kwargs: Keyword arguments for the operation
    """
    start_time = time.time()
    result = operation(*args, **kwargs)
    elapsed = time.time() - start_time
    return result, elapsed

## Generate and Display Initial Data

Create a large dataset and show a sample.

In [4]:
print("Generating large dataset...")
df = generate_large_dataset()
print(f"Generated DataFrame with {len(df)} records and {len(df.columns)} columns")
print("\nSample of data:")
print(df.head())

# Convert the 'timestamp' column to string to avoid serialization issues
df['timestamp'] = df['timestamp'].astype(str)


Generating large dataset...
Generated DataFrame with 100000 records and 5 columns

Sample of data:
                    value category     score  count           timestamp
record_000000  107.051234        C  0.019349      8 2024-01-01 00:00:00
record_000001  100.252799        D  0.913470      9 2024-01-01 00:01:00
record_000002  105.225750        D  0.828215      5 2024-01-01 00:02:00
record_000003   95.292118        C  0.486212      3 2024-01-01 00:03:00
record_000004  110.671849        D  0.723264      1 2024-01-01 00:04:00


## Save to JSONL and Measure Time

Save the dataset to JSONL format and measure the operation time.

In [5]:
print("Saving large dataset to JSONL...")
_, save_time = time_operation(save_jsonldf, 'large_data.jsonl', df)
print(f"Save time: {save_time:.2f} seconds")

Saving large dataset to JSONL...
Save time: 1.01 seconds


## Load and Measure Time

Load the entire dataset and measure the operation time.

In [6]:
print("Loading entire dataset...")
loaded_df, load_time = time_operation(load_jsonldf, 'large_data.jsonl')
print(f"Load time: {load_time:.2f} seconds")

Loading entire dataset...
Load time: 0.58 seconds


In [7]:
loaded_df.head()

,value,category,score,count,timestamp
record_000000,107.051234,C,0.019349,8,2024-01-01 00:00:00
record_000001,100.252799,D,0.913470,9,2024-01-01 00:01:00
record_000002,105.225750,D,0.828215,5,2024-01-01 00:02:00
record_000003,95.292118,C,0.486212,3,2024-01-01 00:03:00
record_000004,110.671849,D,0.723264,1,2024-01-01 00:04:00


## Select Range of Records

Select a range of 1000 records and measure the operation time.

In [8]:
print("Selecting range of 1000 records...")
range_start = 'record_001000'
range_end = 'record_002000'
selected_df, select_time = time_operation(
    select_jsonldf,
    'large_data.jsonl',
    range_start, range_end
)
print(f"Select time: {select_time:.2f} seconds")
print(f"Selected {len(selected_df)} records")

Selecting range of 1000 records...
Select time: 1.25 seconds
Selected 1001 records


In [9]:
selected_df.head()

,value,category,score,count,timestamp
record_001000,110.601815,A,0.078800,2,2024-01-01 16:40:00
record_001001,102.020215,B,0.028568,6,2024-01-01 16:41:00
record_001002,115.741002,B,0.266427,3,2024-01-01 16:42:00
record_001003,109.060197,B,0.080881,6,2024-01-01 16:43:00
record_001004,57.824598,A,0.722577,5,2024-01-01 16:44:00


## Update Batch of Records

Update 1000 random records and measure the operation time.

In [10]:
print("Updating 1000 random records...")
update_indices = [f'record_{i:06d}' for i in range(1000, 2000)]
updates = pd.DataFrame({
    'value': np.random.normal(200, 15, len(update_indices)),
    'category': np.random.choice(['X', 'Y', 'Z'], len(update_indices)),
    'score': np.random.uniform(0, 1, len(update_indices)),
    'count': np.random.poisson(10, len(update_indices)),
    'timestamp': pd.date_range(
        start=datetime(2024, 1, 1),
        periods=len(update_indices),
        freq='1min'
    )
}, index=update_indices)

# Convert timestamp column to string
updates['timestamp'] = updates['timestamp'].astype(str)

_, update_time = time_operation(update_jsonldf, 'large_data.jsonl', updates)
print(f"Update time: {update_time:.2f} seconds")

Updating 1000 random records...
Update time: 0.29 seconds


## Delete Batch of Records

Delete 1000 records and measure the operation time.

In [11]:
print("Deleting 1000 records...")
delete_indices = [f'record_{i:06d}' for i in range(2000, 3000)]
_, delete_time = time_operation(delete_jsonldf, 'large_data.jsonl', delete_indices)
print(f"Delete time: {delete_time:.2f} seconds")

Deleting 1000 records...
Delete time: 0.28 seconds


## Lint the File

Sort and clean the JSONL file, measuring the operation time.

In [12]:
print("Sorting and cleaning the file...")
_, lint_time = time_operation(lint_jsonldf, 'large_data.jsonl')
print(f"Lint time: {lint_time:.2f} seconds")

Sorting and cleaning the file...
Lint time: 2.88 seconds


## Load Final State and Show Statistics

Load the final state of the data and display various statistics.

In [13]:
print("Loading final state...")
final_df = load_jsonldf('large_data.jsonl')
print("\nFinal dataset statistics:")
print(f"Total records: {len(final_df)}")
print("\nNumerical columns summary:")
print(final_df[['value', 'score', 'count']].describe())
print("\nCategory distribution:")
print(final_df['category'].value_counts())

Loading final state...

Final dataset statistics:
Total records: 99000

Numerical columns summary:
              value         score         count
count  99000.000000  9.900000e+04  99000.000000
mean     101.129760  5.007560e-01      5.041556
std       18.022363  2.882981e-01      2.300111
min       34.166920  9.980197e-07      0.000000
25%       90.132025  2.505428e-01      3.000000
50%      100.323619  5.037684e-01      5.000000
75%      110.646522  7.497940e-01      6.000000
max      245.975725  9.999934e-01     18.000000

Category distribution:
B    24817
C    24552
D    24396
A    24235
Y      366
X      327
Z      307
Name: category, dtype: int64


## Performance Summary

Display a summary of all operation timings.

In [14]:
print("\nPerformance Summary:")
print(f"{'Operation':<15} {'Time (s)':<10} {'Records/s':>10}")
print("-" * 35)
print(f"{'Save':<15} {save_time:<10.2f} {len(df)/save_time:>10.0f}")
print(f"{'Load':<15} {load_time:<10.2f} {len(df)/load_time:>10.0f}")
print(f"{'Select':<15} {select_time:<10.2f} {1000/select_time:>10.0f}")
print(f"{'Update':<15} {update_time:<10.2f} {1000/update_time:>10.0f}")
print(f"{'Delete':<15} {delete_time:<10.2f} {1000/delete_time:>10.0f}")
print(f"{'Lint':<15} {lint_time:<10.2f} {len(final_df)/lint_time:>10.0f}")


Performance Summary:
Operation       Time (s)    Records/s
-----------------------------------
Save            1.01            99176
Load            0.58           171512
Select          1.25              801
Update          0.29             3396
Delete          0.28             3601
Lint            2.88            34372


## Cleanup

Remove the JSONL file and its index.

In [15]:
print("\nCleaning up...")
os.remove('large_data.jsonl')
os.remove('large_data.jsonl.idx')
print("Done!")


Cleaning up...
Done!
